In [ ]:
# final-json to feather
import json
import codecs
import os
import pandas as pd

def read_json_file(file_name):
   with codecs.open(file_name,encoding='utf-8') as fin:
       return json.loads(fin.read())

json = read_json_file('/home/marcel/projects/eco/src/python/notebooks/log-final.json')
base_path = json['folder_path']

authors = {}

for key in json['file_descriptors']:
    values = json['file_descriptors'][key]
    rel_path = values['rel_path']
    file_name = values['file_name']
    author_name = values['author_name']
    abs_path = os.path.join(base_path + rel_path, file_name)
    if author_name not in authors:
        authors[author_name] = []
        
    authors[author_name].append(abs_path)
        
# dont start this unless you have some time :)
dataframes = []
# saving dataframes
for key in authors:
    if len(authors[key]) > 20:
        print(key + ' ' + str(len(authors[key])))
        for path in authors[key]:
            print(path)
            df = v3_txt_to_df(path, key)
            dataframes.append(df)
            print(len(dataframes))

print(len(dataframes))
bigdata = pd.concat(dataframes, ignore_index=True)
bigdata.to_feather("parsed_v3_log-final_bigger_than_20.feather")

In [ ]:
# 1

import gensim
from nltk.corpus import stopwords
import numpy as np

model = gensim.models.Word2Vec.load('/mnt/drive1/data/eco/word2vec_models/wiki_plus_v3_valid_combined.txt_numpy.w2vmodel')


# sentence to vector via w2v
def to_vector(sentence):
    sentence = sentence.lower()
    # not using english stop words, only using content words
    words = [word for word in sentence.split() if (word not in stopwords.words("english") and word in model.vocab)]

    vectors = []
    for word in words:
        vectors.append(model[word])

    vector = np.mean(vectors, axis=0)
    return vector


# converts a v4 text file into feather file with w2v vec saved
def v4_txt_to_feather(path, feather_name):
    data = open(path).readlines()

    row_list = []
    for line in data:
        sentence = line.split(';')[0].rstrip()
        vec = to_vector(sentence)
        new_row = {}
        index = 0
        try:
            for i in vec:
                new_row['p'+str(index)] = i
                index += 1

            new_row['sentence'] = sentence
            row_list.append(new_row)
        except:
            # only happening sometimes. \o/
            pass

    df = pd.DataFrame(row_list)
    df.to_feather(feather_name)


def v3_txt_to_df(path, author):
    data = open(path).readlines()

    row_list = []
    for line in data:
        vec = to_vector(line)
        new_row = {}
        index = 0
        try:
            for i in vec:
                new_row['p'+str(index)] = i
                index += 1

            new_row['sentence'] = line
            new_row['author'] = author
            row_list.append(new_row)
        except:
            # only happening sometimes. \o/
            pass

    return pd.DataFrame(row_list)

    
#v4_txt_to_feather('/mnt/drive1/data/eco/NAIL_DATAFIELD_txt/parsed_v4/own_collection_valid.txt', 'out.feather')

In [ ]:
# 2

import feather

df = feather.read_dataframe('parsed_v3_log-final_bigger_than_20.feather')
_df = df

In [ ]:
# 3
# get the closest sentences to this one
import pandas as pd

sentence = 'war power fight pain kill'
sentence_vec = to_vector(sentence)

dist = 0.065

for i in range(300):
    df = df[(df['p'+str(i)] > sentence_vec[i]-dist) & (df['p'+str(i)] < sentence_vec[i]+dist)]
    
pd.options.display.max_colwidth = 500
print(df.sentence)
print(df.author)
# restore old dataframe state
df = _df